# Aprendizado Supervisionado

É o aprendizado em que passamos para o modelo algumas **features** e **labels** em um treinamento. Depois disso, ele consegue inferir **labels** ainda não conhecidas baseadas em novas **features**.


In [ ]:
import pandas
titanic = pandas.read_csv("https://databootcamp.nyc3.digitaloceanspaces.com/titanic_4.csv")
titanic.head()

## Treino e Teste

Para treinar algoritmos de machine learning precisamos dividir nosso dataset entre treino e teste. No treino ele vai aprender o comportamento dos dados para tentar prever no teste as labels. Não podemos passar para o algoritmo todos os dados durante o treinamento porque senão não temos como saber se ele vai conseguir prever dados que ele nunca viu corretamente.

![train_test](https://databootcamp.nyc3.digitaloceanspaces.com/img/Train-Test-Data-Split.png)

Normalmente utilizamos proporção em que 70% do dado vai para o treino e 30% fica para o teste. Para essa divisão o **Sklearn** já tem uma função pronta.

In [ ]:
from sklearn.model_selection import train_test_split

### Vamos determinar que a label seja **Age**

Para essa função já temos que passar separadamente o que é **feature** e o que é **label**. Vamos dizer que a nossa label seja o campo **Age** porque vamos querer prever a melhor forma de preencher ele, mas só podemos pegar as idades preenchidas para treinar.

In [ ]:
titanic_without_na = titanic.dropna()
titanic_without_na.shape

Vamos escolher algumas features que **não** tenham relação com **Age**

In [ ]:
features_cols = [
    "Pclass",
    "Sex_female",
    "SibSp",
    "Parch",
    "Relatives",
    "Embarked_C",
    "Embarked_Q",
    "Embarked_S",
    "Fare",
    "Survived"
]
features = titanic_without_na[features_cols]
label = titanic_without_na["Age"]

In [ ]:
features.head()

In [ ]:
label.head()

Já aplicamos o Scaler somente nas features para evitar que nossos algoritmos de machine learning se confundam quanto as proporções das features

## <font color='blue'>Vocês conseguem aplicar StandardScaler nas features?</font>
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução
features_scaled = features

Normalmente as **features** são chamadas de **X** e as **labels** são chamadas de **Y**

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(
    features_scaled, label, test_size = 0.3, random_state=42
)

Podemos validar os tamanhos das amostras geradas

In [ ]:
x_treino.shape

In [ ]:
x_teste.shape

In [ ]:
y_treino.shape

In [ ]:
y_teste.shape

De quebra ainda temos o conjunto de idade que devemos preencher usando **Regressão Linear**

In [ ]:
titanic_preencher = titanic[pandas.isnull(titanic["Age"])].drop("Age", axis=1)

## Regressão Linear

Método para aproximar duas variáveis linearmente (localização de imóvel vs preço, idade do carro vs preço). Dizemos que duas variáveis tem relação linear se plotarmos seus valores num gráfico e eles "parecerem" uma linha. Os passos para gerar uma regressão linear são 5:

1. randomizar os inputs da função de hipotese
2. computar o erro da predição (Mean Squared Error)
3. Calcular as derivativas parciais
4. Atualiazar os parametros baseados nas derivativas e na taxa de apredizado
5. repetir do 2 ao 4 até o error ser o menor possivel.

#### Função de Hipotese (Hypoteses Function)
$ h0(x) = \theta_1 + \theta_0 == y = m(x) + b$

é a função linear que vimos no primeiro grau da escola, o parâmetro $\theta_1$ (m) define a angulação da linha e o $\theta_0$ (b) define onde a linha cruza o eixo y.
![](https://databootcamp.nyc3.digitaloceanspaces.com/img/regressao_linear.png)

Com o **Sklearn** podemos importar a Regressão Logística já implementada

In [ ]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()

Vamos usar nossos dados de treino para treinar o modelo

In [ ]:
modelo = linear_reg.fit(x_treino, y_treino)

As predições são feitas com o dataset de teste

In [ ]:
predictions = modelo.predict(x_teste)
predictions.shape

Vamos avaliar de forma simples usando o **score** do modelo. Ele usa um método chamado [**r2_score**](https://pt.wikipedia.org/wiki/Coeficiente_de_determina%C3%A7%C3%A3o) que é uma aproximação dos valores para ver se são similares. Esse método varia de 0 a 1.

In [ ]:
modelo.score(x_teste, y_teste)

Bem ruim essa predição... Podemos tentar melhorar ela um pouco

## Seleção de Features

A etapa de seleção de features ajuda a reduzir o overfitting, aumenta a acurácia do modelo e reduz o tempo de treinamento.

### Correlação

Vocês viram bastante sobre a importância de uso das correlações e seus tipos, elas são ótimas para filtrar as melhores colunas para usarmos.

## <font color='blue'>Selecione as melhores features usando correlação</font>
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução

### Recursive Feature Elimination (RFE)

Um algoritmo conhecido para seleção de features que faz uso da correlação por baixo dos panos é o RFE.

Recursivamente vamos removendo uma a uma (ou de *step* em *step*) as features e vendo quais conseguem melhores resultados com os dados através da importância de cada feature no modelo. Conseguimos isso já implementado no **Sklearn**.

In [ ]:
from sklearn.feature_selection import RFE

Podemos escolher o número de passos (step) e o número de features que queremos usar. "Chutei" 6.

In [ ]:
seletor = RFE(linear_reg, n_features_to_select=6, step=1)

Usamos o treino normalmente com **fit**

In [ ]:
features_selected = seletor.fit(x_treino, y_treino)
features_selected

Como escolhemos 6 features, ele faz a melhor combinação de 6 features de acordo com nossos dados. Aonde está **True** significa que ele escolheu aquela feature

In [ ]:
features_selected.support_

Podemos ver um Ranking de quão boas são as features também

In [ ]:
features_selected.ranking_

Com isso temos como ver o *score* que vai transformar o dado para o número de features selecionados e aplicar o *score* do modelo de Regressão Linear

In [ ]:
features_selected.score(x_teste, y_teste)

Podemos automatizar isso para encontrar a melhor quantidade de features e as melhores combinações entre elas

In [ ]:
number_features = features_scaled.shape[1]

In [ ]:
scores = []
linhas = range(1, number_features + 1)
for i in linhas:
    seletor = RFE(linear_reg, n_features_to_select=i, step=1)
    features_selected = seletor.fit(x_treino, y_treino)
    resultado = features_selected.score(x_teste, y_teste)
    scores.append(resultado)

In [ ]:
scores

Vamos usar o *pyploy* para ver esses dados

In [ ]:
from matplotlib import pyplot

In [ ]:
pyplot.plot(linhas, scores)

Conseguimos o melhor número de features em 6. Nossa predição parece continuar horrível... Não acertamos quase nada! A seleção de features serve somente para melhorar um pouco o que o modelo já tinha conseguido, mas normalmente não faz nenhum milagre.

Pelo visto melhor continuar com nossa média dos sexos mesmo e seguir com nossos desafios!

## Vamos tentar prever quem vai sobreviver?

<img src="https://databootcamp.nyc3.digitaloceanspaces.com/img/titanic-naufragio.webp" alt="Drawing" style="width: 700px;"/>

## Regressão Logística

É uma técnica estatística que tem como objectivo produzir, a partir de um conjunto de observações, um modelo que permita a predição de valores tomados por uma variável categórica, frequentemente binária, a partir de uma série de variáveis explicativas contínuas e/ou binárias.

Bem similar a Regressão linear, mas aplicamos uma função **sigmoid** no resultado para determinar em qual categoria se enquadra a classificação.

![sigmoid](https://databootcamp.nyc3.digitaloceanspaces.com/img/sigmoid.png)

No **Sklearn** temos a regressão logísica e vamos usá-la para tentar prever quem morreu ou sobreviveu no Titanic

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42, solver="liblinear")

## <font color='blue'>Agora Precisamos do nosso conjunto de treino e teste para o *Survived*</font>
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução
x_teste = x_treino = titanic.drop(["Survived", "Age"], axis=1)
y_teste = y_treino = titanic["Survived"]

Tendo isso, só realizar o treino assim como na regressão linear

In [ ]:
model = log_reg.fit(x_treino, y_treino)

Depois disso conseguimos fazer as predições com o modelo treinado

In [ ]:
predictions = model.predict(x_teste)
predictions

Temos as predições, mas como avaliar se elas são boas?

## Métricas de qualidade

Devemos saber como medir a qualidade dos nossos modelos. No caso de modelos supervisionados, temos alguns métodos de validação que são bem conhecidos.

### Matriz  de confusão

A Matriz de Confusão é uma forma de mostrar mais claramente quantos são:
- Verdadeiros Positivos (quem morreu e o modelo disse que morreu)
- Falsos Positivos (quem não morreu e o modelo disse que morreu)
- Verdadeiros Negativos (quem não morreu e o modelo disse que não morreu)
- Falsos Negativos (quem morreu e o modelo disse que não morreu)

![tt](https://sebastianraschka.com/images/faq/multiclass-metric/conf_mat.png)

In [ ]:
matriz_confusao = pandas.crosstab(y_teste, predictions) # confusion_matrix
matriz_confusao

### Precision e recall

A partir da matriz de confusão algumas taxas podem ser calculadas.

Vamos visualizar de outra maneira os dados sendo testados:

![pr](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Precis%C3%A3o_e_revoca%C3%A7%C3%A3o.png/262px-Precis%C3%A3o_e_revoca%C3%A7%C3%A3o.png)

Dentro do círculo estão os elementos julgados como 1 (Positivo, nesse caso, sobreviveu).

Fora do círculo estão os elementos que o algoritmo classificou como 0 (Negativo, nesse caso, não-sobreviveu).

O retângulo representa o real valor dos dados.

#### PRECISION

*Precision* representa quanto dos elementos julgados como Sobreviventes, de fato são Sobreviventes. 

Mede a taxa de acerto do classificador dentre os valores <span class="atencao">estimados</span> .

$P=\frac{TP}{TP+FP}$

#### RECALL

*Recall* representa quantos dos reais sobreviventes foram de fato classificados como sobreviventes.

Mede a taxa de acerto do classificador dentre os valores <span class="atencao">reais</span> .

$R=\frac{TP}{TP+FN}$

#### F(Beta)-SCORE

Podemos relacionar o *recall* e *precision* em uma métrica!

![f_beta](https://databootcamp.nyc3.digitaloceanspaces.com/img/f_beta.svg)

Com isso, temos o F1 que é o balanço exato entre *recall* e *precision*

$F1=2\frac{precision  \times  recall}{precision+recall}$

In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_teste, predictions))

Com isso o f1 pode ser acessado através dessa função

In [ ]:
metrics.f1_score(y_teste, predictions)

Que é o mesmo que usar beta=1

In [ ]:
metrics.fbeta_score(y_teste, predictions, beta=1)

## <font color='blue'>Qual valor de Beta prioriza *precision*, 0,5 ou 2? Mostre</font>
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução

## K-Fold Cross Validation

Outra técnica, a *Cross Validation*, ou validação cruzada, é muito usada para estimar a precisão de um modelo. 

A tecnica de *Cross Validation* pode ser usada também para otimização de parâmetros de um algoritmo. 

![cv](https://databootcamp.nyc3.digitaloceanspaces.com/img/cross_val.png)

Com essa técnica passamos todo dataset de features e labels, e ela divide em k tamanhos nosso dado de forma aleatória treinando o modelo.

Vamos Separar as **Features** e as **Labels** pensando no *Survived*

In [ ]:
features = titanic.drop(["Survived", "Age"], axis=1)
labels = titanic["Survived"]

No caso do **Sklearn** o **cv** é o parâmetro **k** de divisões do dataset. Aqui vamos dividir em 10

In [ ]:
from sklearn.model_selection import cross_validate
resultado = cross_validate(log_reg, features, labels, cv=10)
mean_accuracy_test = sum(resultado["test_score"])/len(resultado["test_score"])

print(f"Média do teste: {mean_accuracy_test}")
